# Takes data scraped from Open Broadcast Radio and saves it to a playlist on Spotify

First, read the JSON file:

In [2]:
import json
import os

with open(os.path.join('data', 'openbroadcast-YardThings_2020-04-12-09-59-43.json'), 'r', encoding='utf-8') as f:
    playlist = json.load(f)

Login to Spotify:

In [3]:
import os
import spotify

sp = spotify.login(scope='playlist-read-private playlist-modify-private')
user = sp.current_user()
user['display_name']   # check if login successful



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=ca3831dc54e745ec9fd50425c059da48&response_type=code&redirect_uri=https%3A%2F%2Flocalhost%3A8080%2F&scope=playlist-modify-private+playlist-read-private in your browser




Enter the URL you were redirected to:  https://localhost:8080/?code=AQAsIsdOjY3KuAYULzpDjiNCT66htboj7GmjUbOHBNWwI4x83Ghobvs-ibIlxElYaJS-HillWjj0k9a8S57Dkp_nV5im5R3ydGno0HaUflorM9gY9gVpXKxuKgrTIQTvsp-1vGXGX4JlaTaUgcqiw0eZtH6RXhzD4bVHqWmWDl6cqE6AFXKRohcIIBPkN8aPiv95jU4v_1bPMx8qBbEIHFwMyhRKmNhlHNIVe22a6sZRaJezP9epUlD50Jfp


'Mehmet Birgi'

Get the playlist to edit:

In [4]:
playlist_id, is_new = spotify.get_playlist_by_name(sp, 
        f"OpenBroadcast - {playlist['series']} (by {playlist['author']})", 
        create_if_none=True)
if playlist_id and is_new:
    print(f"Created new playlist with ID {playlist_id}")
elif playlist_id:
    print(f"Found existing playlist with ID {playlist_id}")
else:
    print("Something went wrong :-(")

Created new playlist with ID 2IZoEQC2TAzlsM6JjAVgeS


Add the tracks (skipping duplicates):

In [5]:
tracks_to_add = []
for track in playlist['tracks']:
    q = f"{track['title']} {track['artist']}"
    print(f"searching for: {q}")
    results = sp.search(q, type='track', market='CH')['tracks']['items']
    if results:
        spotify_id = results[0]['id']
        print(f"found: {spotify_id}")
        tracks_to_add.append(spotify_id)
    else:
        print("not found")

searching for: STATION-ID - Roar kurz Roland Widmer
not found
searching for: The Look of Love Dennis Brown
found: 7ewDR6Xct2RYydopHxpqD4
searching for: Alton's Official Daughter Alton Ellis
found: 2HZ76NGX0jXmlu66eTclTj
searching for: Satisfaction Carl Dawkins
found: 20y4i5i8JBwrlre86Om8W9
searching for: STATION-ID - Andavoicepur Roland Widmer
not found
searching for: Santic Rock Leonard Santic All Stars
found: 5HXHUKoo4T8PgQqD25L9JA
searching for: Late at Night William Shakespeare
not found
searching for: Problem #2 Leonard Santic All Stars
found: 6cpuk34sIp6wpzVvCPfnbh
searching for: STATION-ID - Whisper Roland Widmer
not found
searching for: Got to Be Cool Wailing Souls
not found
searching for: Gideons High Max Edwards
found: 6mygVrV8znDiKVlNKjhlHB
searching for: If You Ask Me Leon Dinero
not found
searching for: Bandits Screechy Dan
not found
searching for: STATION-ID - 123 Knock Roland Widmer
not found
searching for: Get Together Carl Dawkins
found: 4m90MX7bkDxo6HwdZf1wqN
searchin

In [6]:
len(tracks_to_add)

162

In [7]:
num_batches = len(tracks_to_add) // 100 + 1  # max 100 tracks per batch
num_batches

2

In [10]:
num_tracks_added = 0
for i in range(0, num_batches):
    print(f"adding batch #{i+1}")
    batch_start = i * 100
    batch_end = len(tracks_to_add) + 1 if (len(tracks_to_add) < batch_start + 100) else batch_start + 100
    print(f"tracks from {batch_start} to {batch_end}")
    spotify.add_tracks(sp, playlist_id, tracks_to_add[batch_start:batch_start+100])
print("finished")

adding batch #1
tracks from 0 to 100
Playlist has 100 existing tracks
Skipping duplicates: True
0 tracks to be added
No tracks added
adding batch #2
tracks from 100 to 163
Playlist has 100 existing tracks
Skipping duplicates: True
57 tracks to be added
57 tracks added
OK
finished
